还在为考试时学生作弊抄答案而烦恼吗？没关系，Python提供私人订制考试题服务，保证每个学生拿到的题目不一样。万恶的Python，妈妈再也不用担心我能作弊抄同学答案了。

这是题库，分为单选题565道、多选题195道、判断题417道和填空题189道。每套题总分100，单选题40分，每题1分；多选题40分，每题2分；判断题和填空题各10分，每题1分。因此需要随机从单选题中抽出40题，多选题抽出20题，判断和填空题中各抽出10题，即可形成一份试卷。
![](images\exam.png)

首先，根据每种题型的数量，在该数量范围内生成不重复随机整数。这些数就是题库中的题目对应的行数。如下，定义了一个具有这样功能的函数。需要传入两个参数，一个是“最大数”`num_max`，决定生成随机数的最大数；另一个是“随机数的个数”`num_qty`，决定生成随机数的个数。比如我们要从417道判断题中随机抽10到，那传入函数的参数就是`Random_num(418, 10)`。为什么最大数是418呢？不是说好的417道题吗？那是因为题库中，第2行开始才是题目，所以在选择行数的时候，最大能选到第418行。那我们生成随机数的时候，也要从2开始，而不是1，因为第一行不是题目。这里用了`while`语句控制随机数的个数，当用于储存随机数的列表`num_list`中整数的个数达到设定的上限时，停止程序。

In [1]:
import random
#不重复随机整数生成函数
def Random_num(num_max, num_qty):
    '''
    num_max:最大数
    num_qty:生成随机数的个数
    '''
    num_list = [] #储存生成的随机数
    while len(num_list) < num_qty: #控制随机数的个数
        num = random.randint(2,num_max) #设定在此范围内取数
        if num in num_list: #判断随机数是否重复
            continue #若重复，则重新生成
        else:
            num_list.append(num) #将不重复的随机数放入列表
    return num_list #生成完成后返回随机数列表

调用以上函数，在2~418这个范围内取10个数，结果如下。

In [2]:
Random_num(418, 10)

[407, 93, 174, 341, 368, 292, 210, 301, 343, 79]

然后定义函数`Question`，按上面生成的整数在题库中抽取对应行的题目。这个函数传入两个参数，一个是`que_type`，用于确定试题类型（"单选题","多选题","判断题","填空题"）。因为不同的题型对应的答案个数可能是不同的，所以需要分别处理。另一个参数是`numbers`，就是题目的编号，来自于上面生成的随机数列表，我们就依据这些数去题库中提取对应编号的题目。然后根据传入的题目类型，去到对应的Excel工作表，按需要抽取对应行的单元格里的数据，将每行的数据存入`single_question`列表，然后再加入总列表`questions`。
<br/>
<br/>对于多选题，因为有些有E选项，有些没有，所以需要加入一个`if`判断语句。只有当E选项对应的单元格不为空，才将其值插入到答案的前面`insert(-1,answerE)`，以保证答案始终在列表的最后面。

In [3]:
#定义函数，按随机数在题库中抽取对应编号的题目
from openpyxl import load_workbook
def Question(que_type,numbers):
    '''
    que_type:试题类型（"单选题","多选题","判断题","填空题"）
    numbers:需要抽取的试题编号
    '''
    questions = [] #储存抽取的题目
    wb = load_workbook("题库.xlsx") #载入题库
    if que_type == "单选题":
        ws = wb[que_type]        
        for i in numbers: #按随机生成的编号抽题
            question = ws["B"+str(i)].value #问题在B列
            answerA = "A：\t" + str(ws["C"+str(i)].value) #选项A在C列，"\t"相当于按一下tab键，在字符间产生间隔
            answerB = "B：\t" + str(ws["D"+str(i)].value) #选项B在D列
            answerC = "C：\t" + str(ws["E"+str(i)].value) #选项C在E列
            answerD = "D：\t" + str(ws["F"+str(i)].value) #选项D在F列
            right_answer  = ws["G"+str(i)].value #正确答案在G列
            single_question = [question, answerA, answerB, answerC, answerD, right_answer] #每行的数据存入列表
            questions.append(single_question) #每个题目的数据存入总列表
    elif que_type == "多选题":
        ws = wb[que_type]        
        for i in numbers:
            question = ws["B"+str(i)].value            
            answerA = "A：\t" + str(ws["C"+str(i)].value)
            answerB = "B：\t" + str(ws["D"+str(i)].value)
            answerC = "C：\t" + str(ws["E"+str(i)].value)
            answerD = "D：\t" + str(ws["F"+str(i)].value)
            right_answer  = ws["H"+str(i)].value
            single_question = [question, answerA, answerB, answerC, answerD, right_answer]
            if ws["G"+str(i)].value: #有些题有E选项，有些没有，因此需要判断一下是否有E选项
                answerE = "E：\t" + str(ws["G"+str(i)].value)
                single_question.insert(-1,answerE) #将E选项插入到答案前面，保持答案是最后一个元素
            questions.append(single_question)
    else: #判断题和填空题，内容只取题干和答案
        ws = wb[que_type]        
        for i in numbers:
            question = ws["B"+str(i)].value
            right_answer  = ws["C"+str(i)].value
            single_question = [question, right_answer]
            questions.append(single_question)
            
    return questions

我们调用函数，查看一下多选题的获取情况，非常满意。有E选项的也处理得很好。

In [5]:
Question("多选题",Random_num(196,20))[10:15]

[['在幻灯片窗格中，选择多个对象可以（   ）。',
  'A：\t单击一个对象后，按住Ctrl键，再单击其他对象',
  'B：\t单击一个对象后，按住Shift键，再单击其他对象',
  'C：\t分别单击每一个要选择的对象',
  'D：\t选择矩形图画工具，利用鼠标在包含这些对象的区域中画一个框',
  'E：\t不使任何画图工具和命令，直接用鼠标在包含这些对象的区域中拖一个框',
  'ABE'],
 ['CPU能直接访问的存储器是（    ）。',
  'A：\tROM',
  'B：\tRAM',
  'C：\tCache',
  'D：\t外存储器',
  'ABC'],
 ['在PowerPoint中，以下叙述正确的有（   ）。',
  'A：\t一个演示文稿中只能有一张应用"标题幻灯片"母板的幻灯片',
  'B：\t在任一时刻，幻灯片窗格内只能查看或编辑一张幻灯片',
  'C：\t在幻灯片上可以插入多种对象，除了可以插入图片、图表外，还可以插入声音、公式和视频等',
  'D：\t备注页的内容与幻灯片内容分别存储在两个不同的文件中',
  'BC'],
 ['在使用PowerPoint 2000的幻灯片放映视图放映演示文稿过程中，要结束放映，可操作的方法有（   ）。',
  'A：\t按Ctrl＋E键',
  'B：\t按回车键',
  'C：\t单击鼠标右键，从弹出的快捷菜单中选中"结束放映"或Esc键',
  'D：\t单击放映屏幕左下角的上三角按钮，在弹出的菜单中选中"结束放映"',
  'E：\t双击放映屏幕左下角的上三角按钮，在弹出的菜单中选中"结束放映"',
  'CD'],
 ['下列有关因特网能够吸引人的原因的说法，正确的是（   ）。',
  'A：\t因特网提供了丰富信息资源',
  'B：\t因特网提供了富有想像力的交流功能',
  'C：\t因特网不具有交互性',
  'D：\t因特网具有及时反馈的特征',
  'E：\t因特网可以给每个人赚大钱',
  'ABD']]

题目抽取好后，就要写入Word文档了。我们先建好了试题模板，把抬头的部分先写好了，后续只需写入题目，并调整一下字体格式就行了。
![](images\temp.png)

建立一个写入函数`To_word`，需要传入两个参数，一个是`number`，用于给考试题编号，比如“考试题1，考试题2...”。另一个参数就是包含试题内容的字典`questions_data`。我们按照“单选题”、“多选题”、“判断题”和“填空题”的顺序分别写入。每种题型的标题单独写为一段，并设置字体加粗，字号加大到12磅。然后再给题目编号，逐个写入。对于多选题，还是要搞下特殊，要用`if`语句判断一下是否有E选项（有E选项的,其列表中的元素才有7个，不然只有6个(`len(i) ==7`)），若有才新建一段写入。最后将考试题按编号命名，并存入“试卷及答案”文件夹。

In [6]:
#写入考试题到word文件
from docx import Document
from docx.shared import Pt #用于设定字体大小（磅值）
from docx.oxml.ns import qn #用于应用中文字体
def To_word(number,questions_data):
    doc = Document("试题-模板.docx")

    #写入单选题
    title1 = "一、单项选择题（共40题，每题1分）"
    p = doc.add_paragraph() #插入段落
    r = p.add_run(title1) #插入文字块
    r.bold = True #字体加粗
    r.font.size = Pt(12) #字号设为12磅
    for index, i in enumerate(questions_data["单选题"],start = 1): #给题目从1开始编号
        doc.add_paragraph(f"{index}. {i[0]}") #题干部分在单独一段
        doc.add_paragraph(f"\t{i[1]}\t\t{i[2]}") #选项A和选项B在同一段落
        doc.add_paragraph(f"\t{i[3]}\t\t{i[4]}") #选项C和选项D在同一段落

    #写入多选题
    title2 = "二、多项选择题（共20题，每题2分）"
    p = doc.add_paragraph()
    r = p.add_run(title2)
    r.bold = True
    r.font.size = Pt(12)
    for index, i in enumerate(questions_data["多选题"],start = 1):
        doc.add_paragraph(f"{index}. {i[0]}")
        doc.add_paragraph(f"\t{i[1]}\t\t{i[2]}")
        doc.add_paragraph(f"\t{i[3]}\t\t{i[4]}")
        if len(i) ==7: #判断是否有E选项，若有，则新建一段落写入
            doc.add_paragraph(f"\t{i[5]}")

    #写入判断题
    title3 = "三、判断题（共10题，每题1分）"
    p = doc.add_paragraph()
    r = p.add_run(title3)
    r.bold = True
    r.font.size = Pt(12)
    for index, i in enumerate(questions_data["判断题"],start = 1):
        doc.add_paragraph(f"\t{index}. {i[0]}")

    #写入填空题
    title4 = "四、填空题（共10题，每题1分）"
    p = doc.add_paragraph()
    r = p.add_run(title4)
    r.bold = True
    r.font.size = Pt(12)
    for index, i in enumerate(questions_data["填空题"],start = 1):
        doc.add_paragraph(f"\t{index}. {i[0]}")

    doc.save(f"试卷及答案\\考试题{number}.docx")

为了方便改试卷的时候参考答案，还得给对应的试题准备一份答案，不然最后老师太受累。我们在提取试题信息的时候就留了一手，将答案也提取在列表里了，只需将其单独写入一个word文件即可。如下是写入答案的函数`Answer`，其中需传入两个参数，一个是`number`，用于在保存文件时给考试题答案编号。另一个参数就是包含答案内容的字典`questions_data`。我们按照“单选题”、“多选题”、“判断题”和“填空题”的顺序分别写入。为了让答案看起来规整，通过`index%10 == 0`设置了每段显示的答案个数。这个意思就是，当每段有10个答案的时候，就执行下面“新建一个段落”的语句`p = doc.add_paragraph()`。由于每种题型的答案的字符串长度不同，这个需要根据实际显示效果做微调。

In [7]:
#写入答案
from docx import Document
from docx.shared import Pt #用于设定字体大小（磅值）
from docx.oxml.ns import qn #用于应用中文字体

def Answer(number,questions_data):
    doc = Document()
    #全局字体设为“宋体”
    doc.styles['Normal'].font.name=u'宋体'
    doc.styles['Normal']._element.rPr.rFonts.set(qn('w:eastAsia'), u'宋体')
    
    title = "计算机系2020第二学期期末考试题(答案)"
    p = doc.add_paragraph()
    r = p.add_run(title)
    r.bold = True
    r.font.size = Pt(20)
    #写入单选题答案
    title1 = "一、单项选择题答案（共40题，每题1分）"
    p = doc.add_paragraph()
    r = p.add_run(title1)
    r.bold = True
    r.font.size = Pt(12)
    
    p = doc.add_paragraph()
    for index, i in enumerate(questions_data["单选题"],start = 1):
        p.add_run(f"{index}. {i[-1]}\t")
        if index%10 == 0: #每段只显示10个答案
            p = doc.add_paragraph() #满10个，则新建段落

    #写入多选题答案
    title2 = "二、多项选择题答案（共20题，每题2分）"
    p = doc.add_paragraph()
    r = p.add_run(title2)
    r.bold = True
    r.font.size = Pt(12)
    p = doc.add_paragraph()
    for index, i in enumerate(questions_data["多选题"],start = 1):
        p.add_run(f"{index}. {i[-1]}\t")
        if index%3 == 0: 
            p = doc.add_paragraph() 
        
    #写入判断题答案
    title3 = "三、判断题答案（共10题，每题1分）"
    p = doc.add_paragraph()
    r = p.add_run(title3)
    r.bold = True
    r.font.size = Pt(12)
    p = doc.add_paragraph()
    for index, i in enumerate(questions_data["判断题"],start = 1):
        p.add_run(f"{index}. {i[-1]}\t")
        if index%5 == 0: #每段只显示5个答案
            p = doc.add_paragraph() #满5个，则新建段落

    #写入填空题
    title4 = "四、填空题答案（共10题，每题1分）"
    p = doc.add_paragraph()
    r = p.add_run(title4)
    r.bold = True
    r.font.size = Pt(12)
    p = doc.add_paragraph()
    for index, i in enumerate(questions_data["填空题"],start = 1):
        p.add_run(f"{index}. {i[-1]}\t\t")
        if index%2 == 0: #每段只显示2个答案
            p = doc.add_paragraph() #满2个，则新建段落
            
    doc.save(f"试卷及答案\\考试题{number}答案.docx")

以上准备工作做好后，就可以开始写主函数，批量生成考试题和对应的答案了。用`for`循环确定需要的不同试卷的套数，这里以20套为例。然后调用`Random_num`生成四组随机数，分别对应四种题型的题目数量。然后将随机数存入字典`question_num`。再调用`Question`函数从题库中抽取对应编号的题目，存入字典`questions_data`。最后调用`To_word`和`Answer`函数，将试题和答案写入并保存为word文件。

In [8]:
#主函数
for number in range(1,21): #不同的试卷数量，此处为20套
    #生成随机题目编号    
    num_single_choice = Random_num(566,40)
    num_mult_choice = Random_num(196,20)
    num_judgment = Random_num(418,10)
    num_completion = Random_num(190,10)
    #将生成的编号存入字典`question_num`
    question_num = {"单选题号":num_single_choice,
           "多选题号":num_mult_choice,
            "判断题号":num_judgment,
            "填空题号":num_completion
           }
    #根据随机生成的题目编号去题库选题，并存入`questions_data`
    questions_data = {
        "单选题":Question("单选题",question_num["单选题号"]),
        "多选题":Question("多选题",question_num["多选题号"]),
        "判断题":Question("判断题",question_num["判断题号"]),
        "填空题":Question("填空题",question_num["填空题号"])
    }
    #将试题写入word文档，并保存
    To_word(number,questions_data)
    #将试题答案写入word文档，并保存
    Answer(number,questions_data)
    print(f"试卷{number}及答案完成！")

试卷1及答案完成！
试卷2及答案完成！


KeyboardInterrupt: 

生成的文件，试题内容和答案内容如下所示，Perfect!
![](images\result_docu.PNG)
![](images\result1.PNG)
![](images\result2.PNG)
除了让每个学生的题目不一样？这个程序还能做什么呢？任你想象。